# Collaborative Topic Regression

In [1]:
import numpy as np
from random import sample

In [4]:
def ctm(R_dict,Theta,n_U,n_epoch=20,batch_size=50,lambda_u=0.002,lambda_v=0.2,alpha=0.002):
    '''
    input: R_dict, {(i,j): Rij} maps ith user and jth item to non-zero rating i,j
           Theta, JxK matrix, jth row is a K-dim topic vector for jth item
           n_U, number of unique users in R_dict
           n_epoch, number of iterations
           lambda_u, regularization parameter for U
           lambda_v, regularization parameter for V
           alpha, learning rate
           
    output: U, a IxK matrix, each row is a K-dim representation for ith user
            V, a JxK matrix, each row is a K-dim representation for jth item
            U_id, user ids
            V_id, item ids
    '''
    n_V,K = Theta.shape # get dimension of parameters
    U = np.random.rand(n_U,K)
    V = Theta.copy() # initialize parameters
    N = len(R_dict)
    pairs = R_dict.keys()
    
    if N<batch_size:
        batch_size = 1
    
    for t in range(n_epoch):
        delta = 0 # change in gradient
        to_use = sample(pairs, batch_size)
        for i,j in to_use: 
                r = R_dict[(i,j)]
                u,v = U[i,:],V[j,:]
                theta = Theta[j,:]
                
                gu = 2*(u.dot(v.T)-r)*v+2*lambda_u*u
                gv = 2*(v.dot(u.T)-r.T)*u+2*lambda_v*(v-theta) # calculate gradient
                u -= alpha*gu
                v -= alpha*gv # update parameters
                
                delta += (np.linalg.norm(gu)+np.linalg.norm(gv)) # calculate change in gradient
                U[i,:] = u
                V[j,:] = v
        
        if delta < 0.001:
            break
    return (U,V)

# example

In [6]:
n = 5
np.random.seed(59)
R = np.random.randint(0,5,size=(n,n))
R_dict = {}
for i in range(n):
    for j in range(n):
        if R[i,j]!=0:
            R_dict[(i,j)] = R[i,j]

Theta = np.random.rand(n,50)

U,V = ctm(R_dict,Theta,n,n_epoch=1000)
np.sum((U.dot(V.T)-R)**2)/len(R_dict)

5.7035805547427936

In [11]:
# To make it converge quick we should take average topic distribution of all the books a costumer has read as his or her initial feature
% timeit U,V = ctm(R_dict,Theta,n,n_epoch=1000)

10 loops, best of 3: 30.6 ms per loop


Estimated Rating

In [9]:
U.dot(V.T).round(1)

array([[ 1.3,  3.9,  2.9,  1.1,  3.9],
       [ 2.7,  5.8,  3. ,  1.7,  4.7],
       [ 1.2,  3.8,  2.2,  1. ,  3. ],
       [ 2.1,  2.1,  2.3,  3.5,  1.1],
       [ 2. ,  5.2,  1.4,  2. ,  3.8]])

True Rating

In [10]:
R

array([[1, 4, 3, 1, 0],
       [3, 0, 3, 1, 0],
       [1, 4, 2, 1, 3],
       [2, 2, 2, 4, 1],
       [2, 0, 1, 2, 0]])

In [330]:
'''
def ctm_matrix(R,Theta,U_batch_size=50,V_batch_size=40,n_epoch=20,lambda_u=0.002,lambda_v=0.2,alpha=0.0002):
    I,J = R.shape
    K = Theta.shape[1] # get dimension of parameters
    
    U = np.random.rand(I,K)
    V = Theta.copy() # initialize parameters
    
    n_U = int(np.ceil(I/(U_batch_size*1.0)))
    n_V = int(np.ceil(J/(V_batch_size*1.0)))# number of batches to be iterated
    
    for t in range(n_epoch):
        delta = 0 # change in gradient
        for u_th in range(n_U):
            for v_th in range(n_V): # for each mini_batch, do
                r = R[U_batch_size*(u_th):U_batch_size*(u_th+1),V_batch_size*(v_th):V_batch_size*(v_th+1)] 
                u = U[U_batch_size*(u_th):U_batch_size*(u_th+1),:]
                v = V[V_batch_size*(v_th):V_batch_size*(v_th+1),:]
                theta = Theta[V_batch_size*(v_th):V_batch_size*(v_th+1),:] # get sub parameters
                
                gu = 2*(u.dot(v.T)-r).dot(v)+2*lambda_u*u
                gv = 2*(v.dot(u.T)-r.T).dot(u)+2*lambda_v*(v-theta) # calculate gradient
                u -= alpha*gu
                v -= alpha*gv # update parameters
                
                delta += (np.linalg.norm(gu)+np.linalg.norm(gv)) # calculate change in gradient
                U[U_batch_size*(u_th):U_batch_size*(u_th+1),:] = u
                V[V_batch_size*(v_th):V_batch_size*(v_th+1),:] = v
        
        if delta < 0.001:
            break
    return (U,V)

'''

'\ndef ctm_matrix(R,Theta,U_batch_size=50,V_batch_size=40,n_epoch=20,lambda_u=0.002,lambda_v=0.2,alpha=0.0002):\n    I,J = R.shape\n    K = Theta.shape[1] # get dimension of parameters\n    \n    U = np.random.rand(I,K)\n    V = Theta.copy() # initialize parameters\n    \n    n_U = int(np.ceil(I/(U_batch_size*1.0)))\n    n_V = int(np.ceil(J/(V_batch_size*1.0)))# number of batches to be iterated\n    \n    for t in range(n_epoch):\n        delta = 0 # change in gradient\n        for u_th in range(n_U):\n            for v_th in range(n_V): # for each mini_batch, do\n                r = R[U_batch_size*(u_th):U_batch_size*(u_th+1),V_batch_size*(v_th):V_batch_size*(v_th+1)] \n                u = U[U_batch_size*(u_th):U_batch_size*(u_th+1),:]\n                v = V[V_batch_size*(v_th):V_batch_size*(v_th+1),:]\n                theta = Theta[V_batch_size*(v_th):V_batch_size*(v_th+1),:] # get sub parameters\n                \n                gu = 2*(u.dot(v.T)-r).dot(v)+2*lambda_u*u\n          

# Spark Version CTM

In [1]:
from __future__ import print_function
import findspark
findspark.init('/Users/Zoe/spark-2.1.0-bin-hadoop2.7/')

import numpy as np
from numpy.random import rand
from numpy import matrix
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

sc = SparkContext('local')
spark = SparkSession(sc)

In [231]:
from pyspark.sql.functions import collect_list, udf
from time import time

simulate data

In [109]:
R = spark.createDataFrame([[0,1,5.0],[0,3,3.0],[0,4,2.0],[1,2,1.0],[1,3,1.0],[2,4,3.0],[3,1,2.0],[3,2,5.0],[4,0,3.0],[4,2,5.0]],['user','item','rating'])
Th = spark.createDataFrame([[0,[0.5,0.5]],[1,[0.3,0.7]],[2,[0.2,0.8]],[3,[0.7,0.3]],[4,[0.4,0.6]]],['item','topic'])

In [186]:
test = spark.createDataFrame([(0, 2, 4.0), (1, 0, 3.0), (2, 0, 2.0)], ["user", "item", "rating"])

training...

In [248]:
def CTM_train(R,Th,I,J,K,LAMBDA,max_iter=10,U_threads=6,V_threads=6):
    '''
    '''
    
    # define update functions
    def updateU(i,v_ind,R,V,LAMBDA):
        '''
        '''
        r = v_ind.shape[0]
        K = V.shape[1]
    
        A = V[v_ind,:].T.dot(V[v_ind,:]) + LAMBDA*r*np.eye(K)
        b = V[v_ind,:].T.dot(R).T
        
        return (np.linalg.solve(A, b)).T
    
    def updateV(j,u_ind,R,U,LAMBDA,Th):
        '''
        '''
        r = u_ind.shape[0]
        K = U.shape[1]
    
        A = U[u_ind,:].T.dot(U[u_ind,:]) + LAMBDA*r*np.eye(K)
        b = U[u_ind,:].T.dot(R).T #+ LAMBDA*r*Th.reshape([K,1])
    
        return (np.linalg.solve(A, b)).T
    
    print('pre-compute block information...')
    U_map = R.groupBy("user").agg(collect_list("item").alias('items'),collect_list("rating").alias('ratings')).sort('user')
    U_map = U_map.repartition(U_threads)
    V_map = R.groupBy("item").agg(collect_list("user").alias('users'),collect_list("rating").alias('ratings')).sort('item').join(Th, "item")
    V_map = V_map.repartition(V_threads)
    
    print('initialize parameters...')
    U = matrix(rand(I,K))
    V = matrix(rand(J,K))
    
    Us = sc.broadcast(U)
    Vs = sc.broadcast(V)
    
    print('update parameters...')
    for i in range(max_iter):
        
        
        st = time()
        U = U_map.rdd.map(lambda r: updateU(r[0],np.array(r[1]),np.array(r[2]),Vs.value,LAMBDA)).reduce(lambda a,b: np.vstack((a,b)))
        Us = sc.broadcast(U)
        
        
        V = V_map.rdd.map(lambda r: updateV(r[0],np.array(r[1]),np.array(r[2]),Us.value,LAMBDA,np.array(r[3]))).reduce(lambda a,b: np.vstack((a,b)))
        Vs = sc.broadcast(V)
        ed = time()
        
        
        print('Finish iteration round: '+str(i)+', use time: '+str(round(ed-st,4))+'s.\n')
    
    return (U,V)        

In [251]:
U1,V1 = CTM_train(R,Th,5,5,2,LAMBDA=0.02,max_iter=10,U_threads=6,V_threads=6)

pre-compute block information...
initialize parameters...
update parameters...
Finish iteration round: 0, use time: 3.3177s.

Finish iteration round: 1, use time: 0.128s.

Finish iteration round: 2, use time: 0.188s.

Finish iteration round: 3, use time: 0.1489s.

Finish iteration round: 4, use time: 0.141s.

Finish iteration round: 5, use time: 0.1298s.

Finish iteration round: 6, use time: 0.1767s.

Finish iteration round: 7, use time: 0.1187s.

Finish iteration round: 8, use time: 0.1367s.

Finish iteration round: 9, use time: 0.1191s.



In [199]:
def CTM_predict(test,U,V):
    '''
    '''
    
    preds = test.rdd.map(lambda r: ((r[0],r[1]),U[r[0],:].dot(V[r[1],:].T)[0,0]))
    return preds

In [203]:
preds = CTM_predict(test,U1,V1)

In [210]:
def evaluate(test,preds,N):
    '''
    '''
    
    se = test.rdd.map(lambda r: ((r[0],r[1]),r[2])).join(preds).map(lambda r: (r[1][0]-r[1][1])**2).reduce(lambda a,b: a+b)
    return se/N

In [212]:
evaluate(test,preds,3)

2.2620496637057741